# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/home/guilherme/anaconda3/INSPER/C_dados/Projeto1-Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'ps5.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,"@jvpmbr we ahem ruim de usar mesmo, não perde ...",0
1,@amandaoufer posso falar como imagino vcs?\nal...,0
2,@ignbrasil @mariananeryf quando sair o trailer...,1
3,@arielsis eu comecei jogar yakuza no meu antig...,0
4,"@psonystas esse ae tá meio doido, tá certo iss...",0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste
0,@rigozfn no ps5 foi 5 gb tô dboa
1,tamanho das atualizações v16.00:\n\nnintendo s...
2,@jhukte eita me dá um ps5 \n\nque eu faço oq v...
3,"@pandaescarlate @brksedu @foioguto caralho, va..."
4,hell let loose será lançado para ps5 em 2021 -...


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O nosso produto é um console de videogame, em sua versão mais atual. Levamos em consideração para definir a relevância dos tweets quesitos como comentários sobre jogos ou funcionalidades do produto, tanto negativas quanto positivas, além de opiniões sobre quesitos que envolvem as vendas desse produto, como locais de venda, preço e principalmente, disponibilidade.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [15]:
import re 


def cleanup(text):
    punctuation = '[!-.:?;@\/|#$%&]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed.lower()
  

In [7]:
# palavras_chave = ['compra', 'jogo', 'jogar', 'comprar', 'trailer', 'caro', 'preço', 'sonho', 'quero']

In [35]:
def tweet_count(tweets):
    x = 0
    palavras_tweets = []
    while x < len(tweets.Treinamento):
        frase = (cleanup(tweets.Treinamento[x]))
        palavras_tweets.extend(frase.split())
        x += 1
    serie_palavras = pd.Series(palavras_tweets)
    tabela_palavras = serie_palavras.value_counts(True)
    return tabela_palavras
tabela_palavras = tweet_count(train)

In [48]:
def calculator_porcent(frase):
    x = 1
    frase = (cleanup(frase))
    probFraseDado = tabela_palavras[frase.split()[0]]
    while x < len(frase.split()):
        probFraseDado *= tabela_palavras[frase.split()[x]]
        x += 1
    return probFraseDado
y = 0
probFraseDados = []
while y < len(train.Treinamento):
    probFraseDados.append(calculator_porcent(train.Treinamento[y]))
    Train.Porcent.append(calculator_porcent(train.Treinamento[y]))
    y += 1
probFraseDados.sort(reverse=True)
probFraseDados[0]

1.9041313070416435e-09

In [38]:
len(train.Treinamento)

300

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**